In [ ]:
# This program use u-net to do binary segmentation on ill skin

In [1]:
%load_ext autoreload
%autoreload 1
%aimport config
from __future__ import division

import tensorflow as tf 
from glob import glob
import numpy as np
import os
from matplotlib import pyplot as plt

import config as cfg 

# from model import u_net

In [20]:
# %load model.py
from __future__ import division

import tensorflow as tf 

import config as cfg 

def Downsampling_block(inputs, in_channels, filters):
    x = conv_group(inputs, in_channels, filters)
    pool_1 = tf.layers.max_pooling2d(x, 2, 2)
    return x, pool_1

def Upsampling_block(input_down, input_up, filters):
    conv_trans = tf.layers.conv2d_transpose(input_up, filters, 2, strides = 2)

#     down_feature_shape = tf.shape(input_down)
#     up_feature_shape = tf.shape(conv_trans)
#     if down_feature_shape[1] % 2 != 0 :
#         # height_pad = down_feature_shape[1].value -up_feature_shape[1].value
#         # width_pad = down_feature_shape[2].value- up_feature_shape[2].value
#         conv_trans = tf.keras.layers.ZeroPadding2D(((0, 1), (0, 0))).apply(conv_trans)
#     if down_feature_shape[2] % 2 != 0:
#         conv_trans = tf.keras.layers.ZeroPadding2D(((0, 0), (0, 1))).apply(conv_trans)

    x = tf.concat([input_down, conv_trans], axis = 3)
    x = conv_group(x, filters)
    return x

def conv_group(inputs, in_channels, filters):
    w = tf.variables(tf.float32, [3, 3, in_channels, filter])
    x = tf.nn.conv2d(inputs, filters, 3,  padding = 'same')
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters, 3,  padding = 'same')
    x = tf.nn.relu(x)
    return x

def u_net(x, y_gt):

    depth = cfg.depth
    filters = cfg.filters_first
    downsample_output = []

    for i in range(depth):
        down_feature_map, x = Downsampling_block(x, filters)
        downsample_output.append(down_feature_map)
        filters = filters * 2

    x = conv_group(x, filters)
    filters = int(filters / 2)

    for i in range(depth):
        x = Upsampling_block(downsample_output[depth - i - 1], x, filters)
        filters = int(filters / 2)

    logits = tf.layers.conv2d(x, 2, 1)

    predictions = tf.argmax(logits, axis = 3)

    loss = tf.losses.softmax_cross_entropy(y_gt, logits)
    return predictions, loss

In [ ]:
# %load model.py
from __future__ import division

import tensorflow as tf 

import config as cfg 

def Downsampling_block(inputs, filters):
    x = conv_group(inputs, filters)
    pool_1 = tf.layers.max_pooling2d(x, 2, 2)
    return x, pool_1

def Upsampling_block(input_down, input_up, filters):
    conv_trans = tf.layers.conv2d_transpose(input_up, filters, 2, strides = 2)

#     down_feature_shape = tf.shape(input_down)
#     up_feature_shape = tf.shape(conv_trans)
#     if down_feature_shape[1] % 2 != 0 :
#         # height_pad = down_feature_shape[1].value -up_feature_shape[1].value
#         # width_pad = down_feature_shape[2].value- up_feature_shape[2].value
#         conv_trans = tf.keras.layers.ZeroPadding2D(((0, 1), (0, 0))).apply(conv_trans)
#     if down_feature_shape[2] % 2 != 0:
#         conv_trans = tf.keras.layers.ZeroPadding2D(((0, 0), (0, 1))).apply(conv_trans)

    x = tf.concat([input_down, conv_trans], axis = 3)
    x = conv_group(x, filters)
    return x

def conv_group(inputs, filters):
    x = tf.nn.conv2d(inputs, filters, 3,  padding = 'same')
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters, 3,  padding = 'same')
    x = tf.nn.relu(x)
    return x

def u_net(x, y_gt):

    depth = cfg.depth
    in_channels = 3
    filters = cfg.filters_first
    downsample_output = []

    for i in range(depth):
        down_feature_map, x = Downsampling_block(x, in_channels, filters)
        downsample_output.append(down_feature_map)
        filters = filters * 2

    x = conv_group(x, filters)
    filters = int(filters / 2)

    for i in range(depth):
        x = Upsampling_block(downsample_output[depth - i - 1], x, filters)
        filters = int(filters / 2)

    logits = tf.layers.conv2d(x, 2, 1)

    predictions = tf.argmax(logits, axis = 3)

    loss = tf.losses.softmax_cross_entropy(y_gt, logits)
    return predictions, loss

In [24]:
def load_image(filename):
    image_string = tf.read_file(filename[0])
    image = tf.image.decode_image(image_string)
    image = tf.image.per_image_standardization(image)
    
    label_string = tf.read_file(filename[1])
    label = tf.image.decode_image(label_string)
    label = tf.cast(label / 255, tf.int8)
    
    return image, label

def generator():

    im_paths = sorted(glob(os.path.join(cfg.data_dir, '*.jpg')))
    gt_paths = sorted(glob(os.path.join(cfg.gt_dir, '*.png')))
    dataset = zip(im_paths, gt_paths)

    split_index = int(len(dataset) * (1 - cfg.val_fraction))
    train_files = dataset[0:split_index]
    val_files = dataset[split_index:]

    train_set = tf.data.Dataset.from_tensor_slices(train_files)
    train_set = train_set.shuffle(buffer_size = split_index)
    train_set = train_set.map(load_image)
    train_set = train_set.batch(cfg.batch_size)
    train_set = train_set.repeat(cfg.epoch)
    train_iter = train_set.make_one_shot_iterator()

    val_set = tf.data.Dataset.from_tensor_slices(val_files)
    val_set = val_set.shuffle(buffer_size = len(dataset) - split_index)
    val_set = val_set.map(load_image)
    val_set = val_set.batch(cfg.batch_size)
    val_set = val_set.repeat(cfg.epoch)
    val_iter = val_set.make_one_shot_iterator()

    train_image, train_label = train_iter.get_next()
    val_image, val_label =val_iter.get_next()

    return train_image, train_label, val_image, val_label

# Testing
# sess = tf.Session()
# batch = sess.run(train_iter.get_next())
# im = batch[0][0]
# gt = batch[1][0] * 255
# print im.shape, gt.shape
# s = np.zeros((gt.shape), dtype = int)
# gt = np.concatenate((gt, s, s), axis = 2)

# plt.imsave('im.jpg',im)
# plt.figure()
# plt.imsave('gt.png', gt)

In [25]:
def train():

    train_image, train_label, val_image, val_label = generator()
    
    print train_image, train_label, val_image, val_label
    
    predictions, loss = u_net(train_image, train_label)
    
    tf.logging.info("Finished loading the network.")

    optimizer = tf.train.GradientDescentOptimizer(cfg.learning_rate)
    train = optimizer.minimize(loss)
    accuracy, _ = tf.metrics.accuracy(train_label, predictions, name = 'my_metric')

    acc_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="my_metric")
    acc_vars_initializer = tf.variables_initializer(var_list=acc_vars)

    init = tf.global_variables_initializer()

    saver = tf.train.Saver()
    sess = tf.Session()
    tf.summary.FileWriter(cfg.visual_dir, sess.graph)

    sess.run(acc_vars_initializer)
    sess.run(init)
    
    while True:
        try:
            loss, accuracy, _ = sess.run((loss, accuracy, train))
            tf.logging.info('loss: {}'.format(loss))
        except tf.errors.OutOfRangeError:
            break

    saver.save(sess, cfg.save_path)

if __name__ == '__main__':
    train()

Tensor("IteratorGetNext_16:0", dtype=float32)
Tensor("IteratorGetNext_16:0", dtype=float32) Tensor("IteratorGetNext_16:1", dtype=int8) Tensor("IteratorGetNext_17:0", dtype=float32) Tensor("IteratorGetNext_17:1", dtype=int8)


TypeError: Expected list for 'strides' argument to 'conv2d' Op, not 3.

In [ ]:
train_writer = tf.python_io.TFRecordWriter(cfg.train_record_path)
val_writer = tf.python_io.TFRecordWriter(cfg.val_record_path)

im_paths = sorted(glob(os.path.join(cfg.data_dir, '*.jpg')))
gt_paths = sorted(glob(os.path.join(cfg.gt_dir, '*.png')))

split_index = int(len(im_paths) * (1 - cfg.val_fraction))
print split_index

# for i in range(len(im_paths)):
for i in range(1):
    im = Image.open(im_paths[i])
    im_data = np.resize(np.array(im) / 255, cfg.im_shape)
    plt.figure()
    plt.imshow(im)
    im_data = im_data.tostring()
    
    test = tf.decode_raw(im_data, tf.float32)
    
    gt = Image.open(gt_paths[i])
    gt_data = np.resize(np.array(gt) / 255, cfg.im_shape[0:-1])
    gt_data = np.array(gt_data, dtype = int)
    gt_data = gt_data.tostring()
    
    images = tf.train.Feature(bytes_list = tf.train.BytesList(value = [im_data]))
    labels = tf.train.Feature(bytes_list = tf.train.BytesList(value = [gt_data]))
    im_shape = tf.train.Feature(int64_list = tf.train.Int64List(value = [cfg.im_shape[0],cfg.im_shape[1],cfg.im_shape[2]]))
    
    features = tf.train.Features(feature = {
        "images" : images,
        "labels" : labels,
        "im_shape" : im_shape
    })
    
    imageFeatures = tf.train.Example(features = features)
    record = imageFeatures.SerializeToString()
    
    if i <= split_index:
        train_writer.write(record)
    else:
        val_writer.write(record)
    
    

In [ ]:
im_paths = sorted(glob(os.path.join(cfg.gt_dir, '*.png')))
im = Image.open(im_paths[0])
plt.imshow(im)
im_data = np.array(im)
print im_data.shape
new = np.array(im_data / 255, dtype = int)

new = new * 255
# plt.figure()
# plt.imshow(new_im)
# # with open('see.txt', 'a') as f:
# #     for i in range(new.shape[0]) :
# #         for j in range(new.shape[1]):
# #             f.write(str(new[i][j]) + ' ')

test = im_data - new
print test.shape
print sum(sum(test))
# #             if im_data[i][j] == 255:
# #                 im_data[i][j] = 0
# # print sum(sum(im_data))
new_im = Image.fromarray(test, mode = 'L')
new_im.save('1.png')
fanren = Image.open('1.png')
fanren = np.array(fanren)
s = fanren
print s.shape
print sum(sum(s))

new = new.reshape((928,1344,1))
s = np.zeros(new.shape, dtype = int)
test = np.concatenate((new, s, s), axis=2)

plt.imsave('qiguai.png', test)


In [5]:
def parseExample(example_ptoto):
    feature_dict = {
        "images" : tf.FixedLenFeature(shape = [], dtype = tf.string), 
        "labels" : tf.FixedLenFeature(shape = [], dtype = tf.string),
        "im_shape" : tf.FixedLenFeature(shape = (3), dtype = tf.int64)
    }
    features = tf.parse_single_example(example_ptoto, feature_dict)
    
#     features = tf.train.example(features)
#     features.ParseFromString(featureString)

    im_shape = features['im_shape']
    print im_shape
    
#     images = tf.string_to_number(features['images'], tf.float32)
    images = tf.decode_raw(features['images'], tf.float32)
    images = tf.reshape(images, im_shape)
#     labels = tf.string_to_number(features['labels'], tf.int32)
    
    labels = tf.decode_raw(features['labels'], tf.int32)
    labels = tf.reshape(labels, im_shape[0:-1])
    
    return images, labels
    
dataset = tf.data.TFRecordDataset(cfg.train_record_path)
dataset = dataset.map(parseExample)
# dataset = dataset.batch(1)

iterator = dataset.make_one_shot_iterator()
# images, labels = iterator.get_next()

# sess = tf.Session()
# sess.run(images)
# print images.shape()

# dataset = dataset.shuffle()
# dataset = dataset.batch(cfg.batch_size)
# dataset = dataset.repeat(cfg.epoch)

# prediction, loss = u_net(dataset)

# optimizer = tf.train.GradientDescentOptimizer(cfg.learning_rate)
# train = optimizer.minimize(loss)
# accuracy, _ = tf.metrics.accuracy(y_gt[:,:,:,1], predictions, name = 'my_metric')


Tensor("ParseSingleExample/Squeeze_im_shape:0", shape=(3,), dtype=int64)
